<a href="https://colab.research.google.com/github/CrissRMFI/TA047R-2C2024-GRUPO08/blob/main/TA047R_TP2_GRUPO08_ENTREGA_N5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANÁLISIS EXPLORATORIO

In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy import displacy

In [ ]:
!wget "https://raw.githubusercontent.com/CrissRMFI/TA047R-2C2024-GRUPO08/refs/heads/main/TP2/test.csv";

--2024-12-01 17:38:03--  https://raw.githubusercontent.com/CrissRMFI/TA047R-2C2024-GRUPO08/refs/heads/main/TP2/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1817926 (1.7M) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]   1.73M  --.-KB/s    in 0.07s   

2024-12-01 17:38:03 (26.2 MB/s) - ‘test.csv’ saved [1817926/1817926]



In [ ]:
!wget "https://raw.githubusercontent.com/CrissRMFI/TA047R-2C2024-GRUPO08/refs/heads/main/TP2/train.csv";

--2024-12-01 17:38:03--  https://raw.githubusercontent.com/CrissRMFI/TA047R-2C2024-GRUPO08/refs/heads/main/TP2/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7301430 (7.0M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   6.96M  --.-KB/s    in 0.09s   

2024-12-01 17:38:04 (75.8 MB/s) - ‘train.csv’ saved [7301430/7301430]



In [ ]:
train_set = pd.read_csv("./train.csv")
test_set = pd.read_csv("./test.csv")

Nos fijamos con que datos estamos trabajando

In [ ]:
train_set.head(5)

,id,title,description,project,storypoint
0,5660,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,3
1,9014,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,5
2,4094,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,5
3,811,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5,2
4,4459,transform processor with script option is broken,Creating the following stream throws exception...,project6,2


In [ ]:
train_set.shape

(7900, 5)

In [ ]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1975 entries, 0 to 1974
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1975 non-null   int64 
 1   title        1975 non-null   object
 2   description  1975 non-null   object
 3   project      1975 non-null   object
dtypes: int64(1), object(3)
memory usage: 61.8+ KB


Verificar si posee datos nulos

In [ ]:
print(train_set.isna().sum() / train_set.shape[0] * 100)

id             0.0
title          0.0
description    0.0
project        0.0
storypoint     0.0
dtype: float64


In [ ]:
print(test_set.isna().sum() / test_set.shape[0] * 100)

id             0.0
title          0.0
description    0.0
project        0.0
dtype: float64


# Pre-Procesamiento

Separamos las variables de la variable target

In [ ]:
X_test_text = test_set[['title','description','project']]

In [ ]:
X_train_text = train_set[['title','description','project']]
Y_train = train_set['storypoint']

In [ ]:
X_train_text.head(5)

,title,description,project
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6
2,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1
3,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5
4,transform processor with script option is broken,Creating the following stream throws exception...,project6


Creamos una funcion que lemmatiza las palabras y nos devuelve las 5 palabras más comunes en un texto. Esto lo vamos a hacer tanto para los titulos como para las descripciones

In [ ]:
nlp = spacy.load("en_core_web_lg")

from collections import Counter
def dejarPalabrasComunes (texto):
  textoTokenizado = nlp(texto)
  palabras = [];
  for token in textoTokenizado:
    if(token.is_alpha and not token.is_stop and not token.is_punct):
      palabras.append(token.lemma_.lower());
  frecuencia = Counter(palabras)
  return frecuencia.most_common(5)

In [ ]:
X_train_token = X_train_text.copy()
X_train_token['palabras descripcion'] = X_train_text['description'].map(lambda x : dejarPalabrasComunes(x))
X_test_token = X_test_text.copy()
X_test_token['palabras descripcion'] = X_test_text['description'].map(lambda x : dejarPalabrasComunes(x))

In [ ]:
X_train_token['palabras titulo'] = X_train_text['title'].map(lambda x : dejarPalabrasComunes(x))
X_test_token['palabras titulo'] = X_test_text['title'].map(lambda x : dejarPalabrasComunes(x))

In [ ]:
X_train_token.head(10)

,title,description,project,palabras descripcion,palabras titulo
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,"[(create, 1), (default, 1), (app, 1), (encount...","[(error, 1), (enable, 1), (appcelerator, 1), (..."
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,"[(commit, 2), (developer, 1), (like, 1), (main...","[(create, 1), (maintenance, 1), (branch, 1)]"
2,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,"[(sam, 2), (api, 1), (gui, 1)]","[(service, 1), (activity, 1), (monitoring, 1),..."
3,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5,"[(mount, 3), (new, 3), (rootfs, 3), (bind, 2),...","[(work, 1), (rootfs, 1), (read, 1)]"
4,transform processor with script option is broken,Creating the following stream throws exception...,project6,"[(create, 2), (stream, 2), (transform, 2), (fa...","[(transform, 1), (processor, 1), (script, 1), ..."
5,Service Registry: REST endpoints URL renaming,Endpoint URLs of Service Registry REST service...,project1,"[(service, 12), (registry, 8), (lookup, 7), (r...","[(service, 1), (registry, 1), (rest, 1), (endp..."
6,TiStudio: Projects - A pane is missing when cr...,Details: If you try to create the default Tita...,project8,"[(project, 3), (ubuntu, 3), (pane, 3), (right,...","[(project, 2), (tistudio, 1), (pane, 1), (miss..."
7,Avoid installing test and performance services...,The membership plans are associated with organ...,project8,"[(membership, 3), (plan, 3), (organization, 3)...","[(avoid, 1), (instal, 1), (test, 1), (performa..."
8,Add Unit Tests for CC SPI infrastructure,"Test Converter, Configuration, Definition and ...",project6,"[(test, 1), (converter, 1), (configuration, 1)...","[(add, 1), (unit, 1), (tests, 1), (cc, 1), (sp..."
9,Agent should not return partial state when a r...,Currently when a user is hitting /state.json o...,project5,"[(agent, 5), (state, 5), (slave, 4), (user, 3)...","[(agent, 1), (return, 1), (partial, 1), (state..."


Creamos un index de palabras que va a funcionar como una bolsa de palabras tanto para el titulo como para la descripción de los storypoints

In [ ]:
indexPalabrasDescripcion= {};
indexPalabrasTitulo= {};

def agregarYDevolverIndex(lista, index, palabraClave):
  indexNuevo = [];
  for elemento in lista:
    if (elemento[0] not in index):
      index[elemento[0]] = str(len(index))+palabraClave
    indexNuevo.append(index[elemento[0]]);
  return indexNuevo;

In [ ]:
palabraClaveTitulo = "_T"
palabraClaveDescripcion= "_D"
X_train_index = X_train_token.copy()
X_train_index['index descripcion'] = X_train_token['palabras descripcion'].map(lambda x :  agregarYDevolverIndex(x, indexPalabrasDescripcion, palabraClaveDescripcion))

In [ ]:
X_test_index = X_test_token.copy()
X_test_index['index descripcion'] = X_test_token['palabras descripcion'].map(lambda x :  agregarYDevolverIndex(x, indexPalabrasDescripcion, palabraClaveDescripcion))

In [ ]:
X_train_index['index titulo'] = X_train_token['palabras titulo'].map(lambda x : agregarYDevolverIndex(x, indexPalabrasTitulo, palabraClaveTitulo))
X_test_index['index titulo'] = X_test_token['palabras titulo'].map(lambda x : agregarYDevolverIndex(x, indexPalabrasTitulo, palabraClaveTitulo))

In [ ]:
X_train_index.head(10)

,title,description,project,palabras descripcion,palabras titulo,index descripcion,index titulo
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,"[(create, 1), (default, 1), (app, 1), (encount...","[(error, 1), (enable, 1), (appcelerator, 1), (...","[0_D, 1_D, 2_D, 3_D, 4_D]","[0_T, 1_T, 2_T, 3_T, 4_T]"
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,"[(commit, 2), (developer, 1), (like, 1), (main...","[(create, 1), (maintenance, 1), (branch, 1)]","[5_D, 6_D, 7_D, 8_D, 9_D]","[5_T, 6_T, 7_T]"
2,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,"[(sam, 2), (api, 1), (gui, 1)]","[(service, 1), (activity, 1), (monitoring, 1),...","[10_D, 11_D, 12_D]","[3_T, 8_T, 9_T, 10_T, 11_T]"
3,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5,"[(mount, 3), (new, 3), (rootfs, 3), (bind, 2),...","[(work, 1), (rootfs, 1), (read, 1)]","[13_D, 14_D, 15_D, 16_D, 17_D]","[12_T, 13_T, 14_T]"
4,transform processor with script option is broken,Creating the following stream throws exception...,project6,"[(create, 2), (stream, 2), (transform, 2), (fa...","[(transform, 1), (processor, 1), (script, 1), ...","[0_D, 18_D, 19_D, 20_D, 21_D]","[15_T, 16_T, 17_T, 18_T, 19_T]"
5,Service Registry: REST endpoints URL renaming,Endpoint URLs of Service Registry REST service...,project1,"[(service, 12), (registry, 8), (lookup, 7), (r...","[(service, 1), (registry, 1), (rest, 1), (endp...","[22_D, 23_D, 24_D, 25_D, 26_D]","[3_T, 20_T, 21_T, 22_T, 23_T]"
6,TiStudio: Projects - A pane is missing when cr...,Details: If you try to create the default Tita...,project8,"[(project, 3), (ubuntu, 3), (pane, 3), (right,...","[(project, 2), (tistudio, 1), (pane, 1), (miss...","[27_D, 28_D, 29_D, 30_D, 31_D]","[24_T, 25_T, 26_T, 27_T, 5_T]"
7,Avoid installing test and performance services...,The membership plans are associated with organ...,project8,"[(membership, 3), (plan, 3), (organization, 3)...","[(avoid, 1), (instal, 1), (test, 1), (performa...","[32_D, 33_D, 34_D, 22_D, 35_D]","[28_T, 29_T, 30_T, 31_T, 3_T]"
8,Add Unit Tests for CC SPI infrastructure,"Test Converter, Configuration, Definition and ...",project6,"[(test, 1), (converter, 1), (configuration, 1)...","[(add, 1), (unit, 1), (tests, 1), (cc, 1), (sp...","[36_D, 37_D, 38_D, 39_D, 40_D]","[32_T, 33_T, 34_T, 35_T, 36_T]"
9,Agent should not return partial state when a r...,Currently when a user is hitting /state.json o...,project5,"[(agent, 5), (state, 5), (slave, 4), (user, 3)...","[(agent, 1), (return, 1), (partial, 1), (state...","[41_D, 42_D, 43_D, 35_D, 44_D]","[37_T, 38_T, 39_T, 40_T, 41_T]"


In [ ]:
print("Hay "+str(len(indexPalabrasTitulo))+" palabras diferentes comunes en los titulos")

Hay 4388 palabras diferentes comunes en los titulos


In [ ]:
print("Hay "+str(len(indexPalabrasDescripcion))+" palabras diferentes comunes en las descripciones")

Hay 4363 palabras diferentes comunes en las descripciones


Convertimos las palabras que incluimos en los index a texto, usando de columnas el index asociado a la palabra

In [ ]:
from collections import Counter

def contar(index, palabras):
    counterPalabras = Counter(palabras)
    counterIndex = Counter({key: 0 for key in index.values()})
    counterIndex.update(counterPalabras)
    return counterIndex

In [ ]:
x_train_indices = X_train_index.copy()
x_train_indices['index description count'] = x_train_indices['index descripcion'].map(lambda x :contar(indexPalabrasDescripcion, x));
x_train_indices =  x_train_indices.join(pd.json_normalize(x_train_indices['index description count']))

In [ ]:
x_test_indices = X_test_index.copy()
x_test_indices['index description count'] = x_test_indices['index descripcion'].map(lambda x :contar(indexPalabrasDescripcion, x));
x_test_indices =  x_test_indices.join(pd.json_normalize(x_test_indices['index description count']))

In [ ]:
x_train_indices['index title count'] = x_train_indices['index titulo'].map(lambda x :contar(indexPalabrasTitulo, x));
x_train_indices =  x_train_indices.join(pd.json_normalize(x_train_indices['index title count']))
x_train_indices = x_train_indices.fillna(0)

In [ ]:
x_test_indices['index title count'] = x_test_indices['index titulo'].map(lambda x :contar(indexPalabrasTitulo, x));
x_test_indices =  x_test_indices.join(pd.json_normalize(x_test_indices['index title count']))
x_test_indices = x_test_indices.fillna(0)

In [ ]:
x_train_indices.head(5)

,title,description,project,palabras descripcion,palabras titulo,index descripcion,index titulo,index description count,0_D,1_D,...,4378_T,4379_T,4380_T,4381_T,4382_T,4383_T,4384_T,4385_T,4386_T,4387_T
0,Error enabling Appcelerator services during ap...,"When creating the default app, I encountered t...",project8,"[(create, 1), (default, 1), (app, 1), (encount...","[(error, 1), (enable, 1), (appcelerator, 1), (...","[0_D, 1_D, 2_D, 3_D, 4_D]","[0_T, 1_T, 2_T, 3_T, 4_T]","{'0_D': 1, '1_D': 1, '2_D': 1, '3_D': 1, '4_D'...",1,1,...,0,0,0,0,0,0,0,0,0,0
1,Create a maintenance branch,"As a developer, I'd like to have a maintenance...",project6,"[(commit, 2), (developer, 1), (like, 1), (main...","[(create, 1), (maintenance, 1), (branch, 1)]","[5_D, 6_D, 7_D, 8_D, 9_D]","[5_T, 6_T, 7_T]","{'0_D': 0, '1_D': 0, '2_D': 0, '3_D': 0, '4_D'...",0,0,...,0,0,0,0,0,0,0,0,0,0
2,Service Activity Monitoring Backend integrated...,SAM API used by SAM GUI,project1,"[(sam, 2), (api, 1), (gui, 1)]","[(service, 1), (activity, 1), (monitoring, 1),...","[10_D, 11_D, 12_D]","[3_T, 8_T, 9_T, 10_T, 11_T]","{'0_D': 0, '1_D': 0, '2_D': 0, '3_D': 0, '4_D'...",0,0,...,0,0,0,0,0,0,0,0,0,0
3,fs::enter(rootfs) does not work if 'rootfs' is...,I noticed this when I was testing the unified ...,project5,"[(mount, 3), (new, 3), (rootfs, 3), (bind, 2),...","[(work, 1), (rootfs, 1), (read, 1)]","[13_D, 14_D, 15_D, 16_D, 17_D]","[12_T, 13_T, 14_T]","{'0_D': 0, '1_D': 0, '2_D': 0, '3_D': 0, '4_D'...",0,0,...,0,0,0,0,0,0,0,0,0,0
4,transform processor with script option is broken,Creating the following stream throws exception...,project6,"[(create, 2), (stream, 2), (transform, 2), (fa...","[(transform, 1), (processor, 1), (script, 1), ...","[0_D, 18_D, 19_D, 20_D, 21_D]","[15_T, 16_T, 17_T, 18_T, 19_T]","{'0_D': 1, '1_D': 0, '2_D': 0, '3_D': 0, '4_D'...",1,0,...,0,0,0,0,0,0,0,0,0,0


Nos deshacemos de las columnas que ya no son necesarias

In [ ]:
x_train_reducido = x_train_indices.drop(["title",	"description", "palabras descripcion","palabras titulo","index descripcion","index titulo","index description count", "index title count"], axis= 1)
x_test_reducido= x_test_indices.drop(["title",	"description","palabras descripcion","palabras titulo","index descripcion","index titulo","index description count", "index title count"], axis= 1)

In [ ]:
x_train_reducido.head(2)

,project,0_D,1_D,2_D,3_D,4_D,5_D,6_D,7_D,8_D,...,4378_T,4379_T,4380_T,4381_T,4382_T,4383_T,4384_T,4385_T,4386_T,4387_T
0,project8,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,project6,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


Nos dimos cuenta que se nos estaba consumiendo la RAM por la cantidad de datasets y la cantidad de columnas que posee, asi que pasamos los dataset a un csv que más adelante va a volver a agarrar para entrenar al modelo

In [ ]:
x_train_reducido.to_csv('x_train_reducido.csv', index=False, header=True)
x_test_reducido.to_csv('x_test_reducido.csv', index=False, header=True)

Liberamos la memoria

In [ ]:
import os
os.kill(os.getpid(), 9)

Volvemos a agarrar la información del dataset

In [ ]:
import numpy as np
import pandas as pd
X_train = pd.read_csv("./x_train_reducido.csv")
X_test= pd.read_csv("./x_test_reducido.csv")

Convertimos la variable categoria "project" a una númerica para que pueda ser tomado por el modelo

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
X_train['project'] = label_encoder.fit_transform(X_train['project'])

In [ ]:
X_train

,project,0_D,1_D,2_D,3_D,4_D,5_D,6_D,7_D,8_D,...,4378_T,4379_T,4380_T,4381_T,4382_T,4383_T,4384_T,4385_T,4386_T,4387_T
0,7,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7895,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7896,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7897,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7898,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_test['project'] = label_encoder.fit_transform(X_test['project'])

# Entrenamiento del Modelo y Predicciones

In [ ]:
Y_train = pd.read_csv("./train.csv")['storypoint']

Realizamos una busqueda de hiperpatrametros para el modelo. La cantidad de folds es 5 debido al coste computacional que supone más folds. Utilizamos la metrica 'neg_root_mean_squared_error' debido a que queremos que no haya grandes desviaciones de los valores reales cuando hacemos la regresión y esta metrica penaliza los errores grandes fuertemente ante la precencia de la raiz cuadrada

In [ ]:
import xgboost as xgb
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

numeroFolds = 5;
parametros = {'objective': ['reg:squarederror'],
              'eval_metric': ['rmse'],
              'max_depth': list(range(3, 15)),
              'learning_rate': [0.01, 0.1, 0.2],
              'subsample':  [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              'n_estimators' : [50,100,150,200],
              }
kFold = KFold(n_splits=numeroFolds)
XGBCBase = xgb.XGBRegressor(random_state=13)
scorer = 'neg_root_mean_squared_error'

randomCv = RandomizedSearchCV(estimator=XGBCBase,
                              random_state=13,
                              param_distributions = parametros,
                              scoring=scorer,
                              cv=kFold,
                              n_jobs=1,
                              n_iter=50
                              )
modelXGBC = randomCv.fit(X_train, Y_train)

Mejores hiperparametros hallados

In [ ]:
modelXGBC.best_params_

{'subsample': 0.7,
 'objective': 'reg:squarederror',
 'n_estimators': 100,
 'max_depth': 12,
 'learning_rate': 0.1,
 'eval_metric': 'rmse'}

In [ ]:
modelXGBC.best_score_

-2.658389944384576

In [ ]:
modeloFinal = modelXGBC.best_estimator_
modeloFinal

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=13, ...)

In [ ]:
Y_train.head(5)

,storypoint
0,3
1,5
2,5
3,2
4,2


Metricas del modelo

In [ ]:
y_train_pred = modeloFinal.predict(X_train)
y_train_pred_rounded = np.round(y_train_pred).astype(int)
print(y_train_pred)

[4.711922  4.273607  2.844212  ... 2.5861332 4.847977  8.537723 ]


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(Y_train, y_train_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Y_train, y_train_pred)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

metricas = {
    'metrica' : ["MSE", "RMSE", "MAE"],
    'valor' :  [mse, rmse, mae]
}
df_metrics = pd.DataFrame(metricas)
print(df_metrics)

  metrica     valor
0     MSE  3.078580
1    RMSE  1.754588
2     MAE  1.318545


Hallamos las predicciones con respecto al set de test y lo pasamos a un archivo csv para poder subirlo a Kaggle

In [ ]:
y_test_pred = modeloFinal.predict(X_test)
print(y_test_pred)

[6.0143785 2.468774  2.1181583 ... 2.2029877 5.527946  4.5745087]


In [ ]:
y_test_pred_rounded = np.round(y_test_pred).astype(int)

In [ ]:
test_set = pd.read_csv("./test.csv")
results = pd.DataFrame({
    'id': test_set['id'],
    'storypoint': y_test_pred_rounded
})

results.head(15)

results.to_csv('results.csv', index=False, header=True)

Crear Pickle

In [ ]:
import pickle

model_pkl_file = "modelo xgboost.pkl"

with open(model_pkl_file, 'wb') as file:
    pickle.dump(modeloFinal, file)